In [16]:
# dim_reduce checks
import yaml

conf_file = "/private/home/marialomeli/faiss_improvements/offline_ivf/config_retro.yaml"
with open(conf_file, "r") as f:
    cfg = yaml.safe_load(f)

# retro 4B
shard_size = 100_000_000  # ~100GB
embeddings_bs = 100_000
bs = shard_size // embeddings_bs  # 1000
nshards = cfg["datasets"]["retro_4B"]["size"] // shard_size + 1
# dt_4B = cfg
print(nshards)

# retro 8B, we added the extra train10.json to retro 4B plus the train11 to train19, each are split in 1024 chunks
shard_size = 100_000_000  # ~100GB
embeddings_bs = 100_000
nshards = cfg["datasets"]["retro_8B"]["size"] // shard_size + 1
nshards

# shard_size and embeddings_bs

41


82

In [14]:
import os
import numpy as np


def read_embeddings(root, filename, file_format, dt, size, input_d):
    fn = f"{root}/{filename}"
    assert os.path.exists(fn)
    if file_format == "raw":
        fl = os.path.getsize(fn)
        nb = fl // input_d // dt.itemsize
        assert nb == size
        assert fl == input_d * dt.itemsize * nb
        return np.memmap(fn, shape=(nb, input_d), dtype=dt, mode="r")
    elif file_format == "npy":
        vecs = np.load(fn, mmap_mode="r")
        assert vecs.shape[0] == size
        assert vecs.shape[1] == input_d
        assert vecs.dtype == dt
        return vecs
    else:
        assert False

In [36]:
root = "/checkpoint/hcir/data/retro-z/repo"
filename = (
    "dataset_name=1t/tokenizer=bert-base-cased/seq_len=2048/chunk_len=64/model=bert-base-cased/train00.jsonl.lz4.npy"
)
format = "npy"
list_of_files = cfg["datasets"]["retro_8B"]["files"]
size = list_of_files[0]["size"]
dt = np.dtype(list_of_files[0]["dtype"])
fn = f"{root}/{filename}"
os.path.exists(f"{root}/{filename}")
# vecs = np.load(fn, mmap_mode='r')
# read_embeddings(root,filename,format,dt,size,768)

ValueError: Cannot load file containing pickled data when allow_pickle=False

In [37]:
[f"train{i:02}.jsonl.lz4.npy" for i in range(10, 11)]

['train10.jsonl.lz4.npy']

In [6]:
import os
import numpy as np

file = "/checkpoint/hcir/data/retro-z/repo/dataset_name=edouard_val/tokenizer=bert-base-cased/seq_len=2048/chunk_len=64/model=bert-base-cased/val.jsonl.lz4.npy"
fl = os.path.getsize(file)
dt = np.dtype(np.float16)
nb = fl // 768 // dt.itemsize
print(768 * dt.itemsize * nb, fl)

18791543808 18791543808


/checkpoint/marialomeli/offline_faiss/test_data/my_data00.npy
/checkpoint/marialomeli/offline_faiss/test_data/my_data01.npy


In [20]:
import numpy as np
import faiss

results = "/checkpoint/marialomeli/offline_faiss/retro/edouard_val_in_retro_4B/I0000000000.npy"
indices = np.load(results)
indices.shape  # we can check if the neighbours are the right number
index = "/checkpoint/marialomeli/offline_faiss/retro/retro_8B/OPQ128_512_IVF4096_PQ128.faissindex"
loaded = faiss.read_index(index)
index_ivf = faiss.downcast_index(faiss.extract_index_ivf(loaded))
index_ivf.nprobe
index_ivf.ntotal
index_ivf.is_trained
index_ivf.code_size

128